In [ ]:
import os
from ultralytics import YOLO
import numpy as np
import json
from PIL import Image

### Create dictionary for image numbers

In [ ]:
filename = 'image_ids.txt'  # replace with your file name

with open(filename, 'r') as f:
    imgnumbers = {}  # create an empty dictionary
    for line in f.readlines():
        line_data = line.split()  # split the line into two values
        key = line_data[0]  # use the first value as the key
        value = line_data[1]  # use the second value as the value
        imgnumbers.update({value: key})  # add the key-value pair to the dictionary


### Crop and stretch test images to ideal format

In [ ]:
def crop_image(image_path, ratio = 1):
    # Open the image
    img = Image.open(image_path)

    # Get the dimensions of the image
    width, height = img.size

    # Find the minimum dimension
    min_dimension = min(width, height)

    # Set the coordinates of the leftmost part of the image to be kept
    left = 0
    top = 1000
    right = min_dimension * ratio
    bottom = min_dimension

    # Crop the image to a square using the leftmost part
    img = img.crop((left, top, right, bottom))

    return img

def stretch_images(input_folder, output_folder):
    # Make sure output_folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through images in input_folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Open image
            input_path = os.path.join(input_folder, filename)
            img = Image.open(input_path)
            
            # Get dimensions
            width, height = img.size
            
            # Calculate new dimensions
            if width < height:
                new_width = height
                new_height = height
            else:
                new_width = width
                new_height = width
            
            # Stretch image
            img = img.resize((new_width, new_height))
            
            # Save image to output_folder
            output_path = os.path.join(output_folder, filename)
            img.save(output_path)

In [ ]:
# Set the input and output directories
input_dir = 'test/images/'
output_dir = 'test/imagescropped/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the input directory
for filename in os.listdir(input_dir):
    # Skip non-image files
    if not filename.endswith('.jpg') and not filename.endswith('.png'):
        continue

    # Crop the image to a square
    img = crop_image(os.path.join(input_dir, filename), ratio=1.3)

    # Save the cropped image to the output directory
    output_filename = os.path.join(output_dir, filename)
    img.save(output_filename)

stretch_images(input_folder=output_dir, output_folder='test/imagescropped_stretched/')

### Load model

In [ ]:
model = YOLO('path_to_model')

### Test prediction on single image

In [ ]:
model.predict(source='path_to_image', show=True)

### Save inference video

In [ ]:
video = model.predict(source='path_to_videofile', show=False, save=True)

### Run inference on entire test set

In [ ]:
results = model.predict(source="test/imagescropped6stretched/", show=False, imgsz=640, stream=True)

In [ ]:
detected_objs = []

counter = 0

for r in results:
    for box in r.boxes:
        bbox = box.xywh[0].tolist()
        bbox[0] = bbox[0] - (bbox[2] / 2)
        bbox[1] = ((bbox[1] - (bbox[3] / 2)) * 0.39245) + 1000 # Rescaling bbox and adding 1000 to the cropped height
        bbox[3] = bbox[3] * 0.39245 # Scaling factor
        detected_objs.append({
            'image_id' : int(imgnumbers[r.path.split('/')[-1]]),
            'bbox' : bbox,
            'category_id' : int(box.cls),
            'id' : counter,
            'score' : float(box.conf)})
        counter += 1

### See number of detected bounding boxes and first bounding box

In [ ]:
print('Number of bounding boxes:' + str(len(detected_objs)))
print('First bounding box:')
print(detected_objs[0])

### Write the bounding boxes to a json file

In [ ]:
json_output = json.dumps(detected_objs)

with open("output.json", "w") as outfile:
    outfile.write(json_output)